https://pytorch.org/audio/stable/tutorials/audio_datasets_tutorial.html#sphx-glr-tutorials-audio-datasets-tutorial-py

In [9]:
import torch
import torchaudio

dataset = torchaudio.datasets.YESNO("./", download=True)

Look at the data

In [102]:
from torchaudio.datasets import YESNO

class ImprovedYESNO(YESNO):
  def __init__(self, transform , *argv,**karg):
    self.transform = transform
    super().__init__(*argv,**karg)
  
  def __getitem__(self,n):
    waveform, sample_rate, label = super().__getitem__(n)
    x = self.transform(waveform)
    return x, torch.tensor(label)

class EqualPad:
  def __init__(self,lenght = 55840):
    self.length = lenght

  def __call__(self, x):
    pad = self.length - x.shape[1]
    pad_left = pad // 2
    pad_right = pad - pad_left
    return torch.nn.functional.pad(x,(pad_left,pad_right))

In [113]:
from torch.utils.data import DataLoader

ds = ImprovedYESNO( transform= EqualPad(), root= "./",download= True)

train, test = torch.utils.data.random_split(ds, [45,15])
train_loader = DataLoader(train, batch_size=8, shuffle=True)
test_loader = DataLoader(test, batch_size=1)


tmp = None
for wav,label in train_loader:
  print(wav[0].shape,)
  tmp =wav

torch.Size([1, 55840])
torch.Size([1, 55840])
torch.Size([1, 55840])
torch.Size([1, 55840])
torch.Size([1, 55840])
torch.Size([1, 55840])


In [160]:
from torch import nn
model = nn.Sequential(
    nn.Conv1d(1,8,200,stride = 16 ),
    nn.ReLU(),
    nn.Conv1d(8,16,100,stride = 8),
    nn.ReLU(),
    nn.Conv1d(16,32,100,stride = 8),
    nn.Flatten(),
    nn.Linear(1312,8),
    nn.Sigmoid()
)
print(tmp.shape)
#print(tmp)
out = model(tmp.float())
print(out.shape)
#model(torch.randn(12,1,55840))

torch.Size([5, 1, 55840])
torch.Size([5, 8])


In [163]:
from sklearn.metrics import accuracy_score
import numpy as np


@torch.inference_mode()
def check(model,data):
  gt = []
  preds = []
  #sigmoid = nn.Sigmoid()
  for wavs,labels in data:
    out = model(wavs)
    #print(out.shape)
    preds.append(out.numpy().round().flatten().astype(int))
    gt.append(labels.numpy().flatten())
    
    #print(out,out.shape,labels.shape)
    #gt.append(labels.flatten())
    #preds.append(model(wavs).flatten().shape)

  #print("====F",np.stack(preds),"===G",np.stack(gt))
  return accuracy_score(np.stack(preds).flatten(),np.stack(gt).flatten())


model.train()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)

for epoch in range(20):
  for wavs,labels in train_loader:
    optimizer.zero_grad()
    out = model(wavs.float())
    loss = criterion(out,labels.float())
    loss.backward()
    optimizer.step()
  print(loss, check(model,train_loader))


ValueError: ignored

align

In [11]:
x = 0
for waveform, sample_rate, label in dataset:
  x = max(x,waveform.shape[1])
print(x)

55840


In [17]:
dataset = torchaudio.datasets.YESNO("./", download=True,transform=lambda sample :  x + sample)
print(dataset[2][0].shape)

TypeError: ignored

In [ ]:
for waveform, sample_rate, label in dataset:
    torch.zeros(x)

In [ ]:
transform = 

In [8]:
from torchaudio.datasets import SPEECHCOMMANDS

ds = SPEECHCOMMANDS("/content",download = True)

  0%|          | 0.00/2.26G [00:00<?, ?B/s]

KeyboardInterrupt: ignored